In [2]:
# 1
# Chance Wiese

import pandas as pd
import numpy as np
from datetime import datetime
import psycopg2
import pymssql

psycopg2_conn = psycopg2.connect(
    database="puptrail", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432')
psycopg2_cursor = psycopg2_conn.cursor()

pymssql_conn = pymssql.connect(
    server='stairway.usu.edu',
    user='5330user',
    password='pipelinesnow',
    database='pup_sl')
pymssql_cursor = pymssql_conn.cursor()

In [3]:
# 2
print('''
Name: Puptrail, Inc.
Product: “teardrop” camper trailers
Locations:
    1. Salt Lake City
    2. Phoenix
    3. Boise
''')


Name: Puptrail, Inc.
Product: “teardrop” camper trailers
Locations:
    1. Salt Lake City
    2. Phoenix
    3. Boise



In [4]:
# 3
query = '''
SELECT 
    (o.builtdate - o.promisedate) AS delay_days,
    COUNT(o.*) AS num_trailers,
    AVG(s.delsatisfac) AS avg_satisfaction
FROM oline o
JOIN salesorder s ON o.orderid = s.orderid
JOIN product p ON o.productid = p.prodid
JOIN prodcat pc ON p.catid = pc.catid
WHERE pc.category = 'Trailers'
AND o.builtdate > o.promisedate
GROUP BY delay_days
ORDER BY delay_days;
'''

df = pd.read_sql(query, psycopg2_conn)
print("Delay Info:")
print(df)

df_corr = df.corr()
print("\nCorrelation Matrix:")
print(df_corr)

print("\nRelationship:")
print('''There is a slight positive relationship between delay days and average satisfaction (correlation = 0.09).
This tells us that customers are generally understanding of delays and aren't too unhappy with them.
There are generally less trailers as delay days increase, which is good, but may also skew satisfaction.
Where there are about 8-14 days delayed, more trailers are present and average satisfaction does drop, 
so it should be noted that with more trailers (and reviews), satisfaction does drop with more delay days.''')

C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\2980103058.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, psycopg2_conn)


Delay Info:
    delay_days  num_trailers  avg_satisfaction
0            1             6          4.333333
1            2             6          3.250000
2            3            12          3.583333
3            4             9          3.571429
4            5             9          4.285714
5            6            14          3.500000
6            7             7          4.000000
7            8            17          3.642857
8            9            16          4.692308
9           10            13          2.666667
10          11            16          3.733333
11          12            11          3.818182
12          13             8          3.750000
13          14            13          3.818182
14          16             4          4.250000
15          17             2          3.000000
16          18             2          4.500000
17          19             3          4.000000

Correlation Matrix:
                  delay_days  num_trailers  avg_satisfaction
delay_days   

In [5]:
# 4
query = '''
WITH trailer_sales AS (
    SELECT 
        sp.spid,
        sp.spfname,
        sp.splname,
        si.sitename,
        COUNT(DISTINCT CASE WHEN pc.category = 'Trailers' THEN ol.orderid END) AS trailers_sold,
        COUNT(DISTINCT CASE WHEN pc.category = 'Upgrades' THEN ol.olid END) AS upgrades_sold,
        SUM(CASE WHEN pc.category = 'Upgrades' THEN ol.linetotal ELSE 0 END) AS upgrade_revenue
    FROM salesperson sp
    JOIN site si ON sp.siteid = si.siteid
    JOIN salesorder so ON sp.spid = so.sellerid
    JOIN oline ol ON so.orderid = ol.orderid
    JOIN product p ON ol.productid = p.prodid
    JOIN prodcat pc ON p.catid = pc.catid
    WHERE sp.termdate IS NULL
    GROUP BY sp.spid, sp.spfname, sp.splname, si.sitename
)
SELECT 
    spfname,
    splname,
    sitename,
    trailers_sold,
    upgrades_sold,
    upgrade_revenue,
    CAST(upgrade_revenue / trailers_sold AS NUMERIC(10,2)) AS avg_upsell_per_trailer
FROM trailer_sales
ORDER BY avg_upsell_per_trailer DESC;
'''

df = pd.read_sql(query, psycopg2_conn)
print("Upsell Info by Salesperson:")
print(df)

Upsell Info by Salesperson:
    spfname     splname        sitename  trailers_sold  upgrades_sold  \
0    Maryla    Kunibert  Salt Lake City            276            335   
1   Milburn       Guido  Salt Lake City            309            361   
2     Kelan      Fowler           Boise            206            212   
3    Shayan      Hewitt           Boise            291            320   
4   Alberto       Dimas  Salt Lake City            112            121   
5    Archie      Dinesh  Salt Lake City            171            175   
6   Roksana     Antonia           Boise            297            235   
7     Andre       Clare         Phoenix            200             93   
8     Robby        Lexy         Phoenix            141             54   
9    Bethel      Derick         Phoenix            131             46   
10     Miah  Wittington         Phoenix            131             31   

    upgrade_revenue  avg_upsell_per_trailer  
0          287825.0                 1042.84  
1  

C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\556694782.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, psycopg2_conn)


In [6]:
# 5
# CustOrder
query = '''
SELECT 
    COLUMN_NAME,
    DATA_TYPE,
    CHARACTER_MAXIMUM_LENGTH as MaxChar,
    NUMERIC_PRECISION,
    IS_NULLABLE
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_NAME = 'CustOrder';
'''
df = pd.read_sql(query, pymssql_conn, index_col='COLUMN_NAME')
print("CustOrder:")
print(df)

# Order_Line
query = '''
SELECT 
    COLUMN_NAME,
    DATA_TYPE,
    CHARACTER_MAXIMUM_LENGTH as MaxChar,
    NUMERIC_PRECISION,
    IS_NULLABLE
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_NAME = 'Order_Line';
'''
df = pd.read_sql(query, pymssql_conn, index_col='COLUMN_NAME')
print("\nOrder_Line:")
print(df)

# Inventory
query = '''
SELECT 
    COLUMN_NAME,
    DATA_TYPE,
    CHARACTER_MAXIMUM_LENGTH as MaxChar,
    NUMERIC_PRECISION,
    IS_NULLABLE
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_NAME = 'Inventory';
'''
df = pd.read_sql(query, pymssql_conn, index_col='COLUMN_NAME')
print("\nInventory:")
print(df)

CustOrder:
            DATA_TYPE  MaxChar  NUMERIC_PRECISION IS_NULLABLE
COLUMN_NAME                                                  
orderid           int      NaN               10.0          NO
odate            date      NaN                NaN          NO
custname      varchar     50.0                NaN          NO
custphone     varchar     20.0                NaN         YES
custemail     varchar    100.0                NaN         YES
contactpref   varchar     10.0                NaN         YES
custstate        char      2.0                NaN         YES
pmtid         varchar     20.0                NaN         YES
saletotal       money      NaN               19.0         YES
amtpaid         money      NaN               19.0         YES
promisedate      date      NaN                NaN         YES
salesperson   varchar    100.0                NaN          NO


C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\2349154933.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pymssql_conn, index_col='COLUMN_NAME')



Order_Line:
            DATA_TYPE  MaxChar  NUMERIC_PRECISION IS_NULLABLE
COLUMN_NAME                                                  
olid              int      NaN               10.0          NO
orderid           int      NaN               10.0          NO
prodname      varchar     50.0                NaN          NO
qty           tinyint      NaN                3.0          NO
linetotal       money      NaN               19.0          NO

Inventory:
            DATA_TYPE  MaxChar  NUMERIC_PRECISION IS_NULLABLE
COLUMN_NAME                                                  
ProdName     nvarchar    255.0                NaN          NO
Price           money      NaN               19.0         YES
LeadTime          int      NaN               10.0         YES
Upgrade          char      1.0                NaN         YES
CreateStamp  datetime      NaN                NaN         YES
UpdateStamp  datetime      NaN                NaN         YES


C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\2349154933.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pymssql_conn, index_col='COLUMN_NAME')
C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\2349154933.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pymssql_conn, index_col='COLUMN_NAME')


In [7]:
# 6
# Add new columns to SalesOrder table
query = '''
ALTER TABLE salesorder 
ADD COLUMN IF NOT EXISTS TxID INT,
ADD COLUMN IF NOT EXISTS Tx_IngestTimestamp TIMESTAMP;
'''
psycopg2_cursor.execute(query)
psycopg2_conn.commit()

# Verify the changes
query = '''
SELECT orderid, odate, ototal, txid, tx_ingesttimestamp
FROM salesorder
ORDER BY odate
LIMIT 10;
'''
df = pd.read_sql(query, psycopg2_conn)
print(df)

C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\322494862.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, psycopg2_conn)


   orderid       odate    ototal  txid tx_ingesttimestamp
0    28449  2022-01-01  25083.90  None               None
1    28450  2022-01-01  26504.80  None               None
2    28451  2022-01-01  19439.00  None               None
3    28452  2022-01-01  17995.00  None               None
4    28453  2022-01-01     14.85  None               None
5    28454  2022-01-01  26227.00  None               None
6    28455  2022-01-01  18782.90  None               None
7    28456  2022-01-01  18444.00  None               None
8    28457  2022-01-01     24.75  None               None
9    28448  2022-01-01     88.90  None               None


In [8]:
# 7
# Create sequences
psycopg2_cursor.execute(''' 
CREATE SEQUENCE IF NOT EXISTS custid_sequence;
SELECT setval('custid_sequence', (SELECT MAX(custid) FROM customer));
CREATE SEQUENCE IF NOT EXISTS orderid_sequence;
SELECT setval('orderid_sequence', (SELECT MAX(orderid) FROM salesorder));
''')
psycopg2_conn.commit()

sites = ['pup_sl', 'pup_bo', 'pup_ph']
for site in sites:
    site_conn = pymssql.connect(
        server='stairway.usu.edu',
        user='5330user',
        password='pipelinesnow',
        database=site
    )
    site_cursor = site_conn.cursor()
    query = "SELECT * FROM custorder;"
    df = pd.read_sql(query, site_conn)
    columns = ['orderid', 'odate', 'custname', 'custphone', 'custemail', 'contactpref', 'custstate', 'pmtid', 'saletotal', 'amtpaid', 'promisedate', 'salesperson']
    
    for x in df.index:
        orderid, odate, custname, custphone, custemail, contactpref, custstate, pmtid, saletotal, amtpaid, promisedate, salesperson = [df[column].loc[x] for column in columns]
        promisedate_val = f"'{promisedate}'" if promisedate is not None else 'NULL'
        
        psycopg2_cursor.execute("SELECT nextval('custid_sequence');")
        new_custid = psycopg2_cursor.fetchone()[0]
        
        # current_customers_df = pd.read_sql("SELECT CONCAT(custfname, ' ', custlname) AS full_name FROM customer;", psycopg2_conn)
        # current_customers_list = current_customers_df['full_name'].tolist()
        # Realized I didn't need this
        
        custfname, custlname = custname.split(' ')
        customer_query = f'''
            INSERT INTO customer (custid, custfname, custlname, custphone, custemail, custstate, contactpref, pmtid, addedstamp)
            VALUES ({new_custid}, '{custfname}', '{custlname}', '{custphone}', '{custemail}', '{custstate}', '{contactpref}', {pmtid}, '{datetime.now()}');
        '''
        # print("Insert customer query:\n", customer_query)
        psycopg2_cursor.execute(customer_query)
        
        psycopg2_cursor.execute("SELECT nextval('orderid_sequence');")
        new_orderid = psycopg2_cursor.fetchone()[0]
        
        site_id = {'pup_sl': 1, 'pup_bo': 2, 'pup_ph': 3}[site]
        psycopg2_cursor.execute(f"SELECT spid FROM salesperson WHERE CONCAT(spfname, ' ', splname) = '{salesperson}';")
        sellerid = psycopg2_cursor.fetchone()[0]
        
        
        order_query = f'''
            INSERT INTO salesorder (orderid, odate, custid, sellerid, ototal, ordsatisfac, delsatisfac, txid, tx_ingesttimestamp)
            VALUES ({new_orderid}, '{odate}', {new_custid}, {sellerid}, {saletotal}, NULL, NULL, {orderid}, '{datetime.now()}');
        '''
        # print("Insert salesorder query:\n", order_query)
        psycopg2_cursor.execute(order_query)
        
        ol_query = f"SELECT * FROM order_line WHERE orderid = {orderid};"
        ol_df = pd.read_sql(ol_query, site_conn)
        
        for y in ol_df.index:
            prodname = ol_df['prodname'].loc[y]
            qty = ol_df['qty'].loc[y]
            linetotal = ol_df['linetotal'].loc[y]
            
            psycopg2_cursor.execute(f"SELECT prodid FROM product WHERE prodname = '{prodname}';")
            productid = psycopg2_cursor.fetchone()[0]
            
            unitprice = linetotal / qty
            oline_query = f'''
                INSERT INTO oline (orderid, productid, lineqty, lineprice, linetotal, promisedate, builtdate, deldate)
                VALUES ({new_orderid}, {productid}, {qty}, {unitprice}, {linetotal}, {promisedate_val}, NULL, NULL);
            '''
            # print("Insert oline query:\n", oline_query)
            psycopg2_cursor.execute(oline_query)
        
        psycopg2_conn.commit()
        site_name = site.split('_')[1].upper()
        print(f"Order {orderid} from {site_name} recorded as {new_orderid}.")

C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\588263925.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, site_conn)
C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\588263925.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ol_df = pd.read_sql(ol_query, site_conn)


Order 27 from SL recorded as 30979.
Order 28 from SL recorded as 30980.
Order 29 from SL recorded as 30981.
Order 30 from SL recorded as 30982.
Order 31 from SL recorded as 30983.
Order 32 from SL recorded as 30984.
Order 33 from SL recorded as 30985.
Order 34 from SL recorded as 30986.
Order 35 from SL recorded as 30987.
Order 36 from SL recorded as 30988.
Order 37 from SL recorded as 30989.
Order 38 from SL recorded as 30990.
Order 39 from SL recorded as 30991.
Order 40 from SL recorded as 30992.
Order 41 from SL recorded as 30993.
Order 42 from SL recorded as 30994.
Order 43 from SL recorded as 30995.
Order 44 from SL recorded as 30996.
Order 45 from SL recorded as 30997.
Order 46 from SL recorded as 30998.
Order 47 from SL recorded as 30999.
Order 13 from BO recorded as 31000.
Order 14 from BO recorded as 31001.
Order 15 from BO recorded as 31002.
Order 16 from BO recorded as 31003.
Order 17 from BO recorded as 31004.
Order 18 from BO recorded as 31005.
Order 19 from BO recorded as

In [9]:
# 8
query = '''
SELECT 
    so.orderid,
    so.odate,
    SUM(ol.lineqty) as total_qty,
    so.ototal,
    sp.siteid,
    c.custlname
FROM salesorder so
JOIN customer c ON so.custid = c.custid
JOIN oline ol ON so.orderid = ol.orderid
JOIN salesperson sp ON so.sellerid = sp.spid
WHERE DATE(so.tx_ingesttimestamp) = CURRENT_DATE
GROUP BY so.orderid, so.odate, so.ototal, sp.siteid, c.custlname
ORDER BY so.odate, c.custlname;
'''

df = pd.read_sql(query, psycopg2_conn, index_col='orderid')
print(df)

C:\Users\Chanc\AppData\Local\Temp\ipykernel_33236\3273112721.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, psycopg2_conn, index_col='orderid')


              odate  total_qty    ototal siteid   custlname
orderid                                                    
31013    2022-08-28          7  30921.00     PH   Henderson
30980    2022-08-28          2  25990.00     SL  Montgomery
31000    2022-08-28          4  26518.00     BO      Sawyer
30979    2022-08-28          1  24995.00     SL   Schroeder
30981    2022-08-28          6  29777.90     SL        Ward
31014    2022-08-29          1  24995.00     PH      Fields
31001    2022-08-29          7  20698.85     BO    Garrison
31015    2022-08-29          8  19537.80     PH     Hammond
30982    2022-08-29          5    114.80     SL        Hill
30983    2022-08-30          4     19.80     SL      Dorsey
31003    2022-08-30          2      9.90     BO       Drake
30984    2022-08-30          7  18024.70     SL      Greene
31002    2022-08-30          1     79.00     BO   Gutierrez
31016    2022-08-30          5     24.75     PH        Hall
30985    2022-08-30          4     19.80

In [10]:
psycopg2_cursor.close()
psycopg2_conn.close()

site_cursor.close()
site_conn.close()

pymssql_cursor.close()
pymssql_conn.close()